# Fetch Assessment

In [1]:
from utils import *
from dataset import *

In [2]:
dataset = TaskDataset(data=None)

Using cache found in C:\Users\yashv/.cache\torch\hub\huggingface_pytorch-transformers_main


# Task 1
Sentence Embedding Model, architecture:
 - Embedding Layer 
 - Positional Encoding
 - Transformer Encoder Layer
 - Mean Pooling
 - Linear Layer (generates embedding)

In [3]:
# initialize models
n_layers = 1 # The number of transformer encoder layers
d_model = 512 # The input embedding dimension
embed_size = 300 # The output embedding size
nhead = 8 # The number of attention heads in encoder layer
task_type = "se" # The type of task (sentence embedding or sentiment classification)
vocab_size = dataset.vocab_size # The size of the vocabulary (using hugging face bert tokenizer)
base_model, task_model = init_model(vocab_size, n_layers, d_model, nhead, embed_size, task_type)

In [4]:
# Generate sample input
input_data = torch.randint(0, dataset.vocab_size, (2, 100))

# Forward pass
base_output = base_model(input_data)
task_output = task_model(base_output)

print("Base model output:", base_output.shape)
print("Task model output:", task_output.shape)

# Generate input based on text
text1 = "How, are you?"
text2 = "I am good"

input_data1 = dataset.encode(text1)
input_data2 = dataset.encode(text2)

# Forward pass
base_output1 = base_model(input_data1)
base_output2 = base_model(input_data2)
task_output1 = task_model(base_output1)
task_output2 = task_model(base_output2)

print("Base model output 1:", base_output1.shape)
print("Base model output 2:", base_output2.shape)
print("Task model output 1:", task_output1.shape)   
print("Task model output 2:", task_output2.shape)

Base model output: torch.Size([2, 100, 512])
Task model output: torch.Size([2, 300])
Base model output 1: torch.Size([1, 7, 512])
Base model output 2: torch.Size([1, 5, 512])
Task model output 1: torch.Size([1, 300])
Task model output 2: torch.Size([1, 300])
